# Prepare interpret-cxr dataset

In [ ]:
import datasets
from datasets import load_dataset

Download

data will be cached to `~.cache/huggingface/datasets` by default

In [ ]:
dataset = load_dataset("StanfordAIMI/interpret-cxr-public")

In [ ]:
dataset_test = load_dataset("StanfordAIMI/interpret-cxr-test-hidden")

In [ ]:
dataset_test_public = load_dataset("StanfordAIMI/interpret-cxr-test-public")

Save to disk

In [ ]:
dataset.save_to_disk("/home/yuxiang/liao/mimic/interpret-cxr")

In [ ]:
dataset_test.save_to_disk("/home/yuxiang/liao/mimic/interpret-cxr-test-hidden")

In [ ]:
dataset_test_public.save_to_disk("/home/yuxiang/liao/mimic/interpret-cxr-test-public")

Load from disk

In [ ]:
dataset = datasets.load_from_disk("/home/yuxiang/liao/mimic/interpret-cxr")

In [ ]:
dataset["train"][0]["images"]

In [ ]:
dataset.add(dataset_test_public)

# Resize mimic-cxr-jpg dataset

In [ ]:
import os
from tqdm import tqdm
from PIL import Image

In [ ]:
def resize_images(input_dir, output_dir, max_size=768):
    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)

    # 遍历输入目录下的所有文件和子目录
    for root, dirs, files in tqdm(os.walk(input_dir)):
        for filename in files:
            # 检查文件是否为图片文件
            if filename.lower().endswith((".png", ".jpg", ".jpeg", ".gif", ".bmp")):
                # 构建输入文件的完整路径
                input_path = os.path.join(root, filename)

                # 构建输出文件的相对路径（保留相对目录结构）
                relative_path = os.path.relpath(input_path, input_dir)
                output_path = os.path.join(output_dir, relative_path)

                # 创建输出文件所在的目录（如果不存在）
                os.makedirs(os.path.dirname(output_path), exist_ok=True)

                # 打开图片文件并进行等比例缩放
                with Image.open(input_path) as img:
                    # 获取原始图像的宽度和高度
                    width, height = img.size

                    # 计算缩放比例
                    if max(width, height) > max_size:
                        if width >= height:
                            new_width = max_size
                            new_height = int(height * max_size / width)
                        else:
                            new_width = int(width * max_size / height)
                            new_height = max_size
                        # 缩放图像
                        img = img.resize((new_width, new_height), Image.BICUBIC)

                    # 保存缩放后的图像到输出路径
                    img.save(output_path)


# 指定输入目录和输出目录
input_directory = "/home/yuxiang/liao/mimic/mimic-cxr-jpg/2.1.0/files"
output_directory = "/home/yuxiang/liao/mimic/mimic-cxr-jpg-resized/files"

# 调用函数进行图片缩放
resize_images(input_directory, output_directory)

In [ ]:
import shutil
import os

input_directory = "/home/yuxiang/liao/mimic/mimic-cxr-jpg/2.1.0"
output_directory = "/home/yuxiang/liao/mimic/mimic-cxr-jpg-resized"

# Copy the file to the destination directory
shutil.copy(os.path.join(input_directory, "train_mimic.json"), output_directory)
shutil.copy(os.path.join(input_directory, "val_mimic.json"), output_directory)

# Prepare mimic-cxr dataset

- https://stanford-aimi.github.io/RRG24/
- https://github.com/Stanford-AIMI/RRG24/issues/3

In [ ]:
from datasets import load_dataset, Sequence, Image, DatasetDict, concatenate_datasets
import os

In [ ]:
data_dir = "/home/yuxiang/liao/mimic/mimic-cxr-jpg-resized"

dataset_mimic = load_dataset("json", data_files={"train": os.path.join(data_dir, "train_mimic.json"), "validation": os.path.join(data_dir, "val_mimic.json")})

In [ ]:
PATH_PREFIX = data_dir


def add_prefix(example):
    example["images"] = [os.path.join(PATH_PREFIX, i) for i in example["images"]]
    return example


dataset_mimic = dataset_mimic.map(add_prefix, num_proc=8)

In [ ]:
dataset_mimic = dataset_mimic.cast_column("images", Sequence(Image()))

In [ ]:
dataset_mimic["train"][1]["images"]

# Concat both datasets

In [ ]:
dataset_final = DatasetDict({"train": concatenate_datasets([dataset["train"], dataset_mimic["train"]]), "validation": concatenate_datasets([dataset["validation"], dataset_mimic["validation"]])})

In [ ]:
dataset_final